<a href="https://colab.research.google.com/github/DennisMuturi-prog/Grocery_ingredient_generator/blob/main/recipe_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
from gensim.models import Word2Vec

In [15]:
model2 = Word2Vec.load("/content/drive/MyDrive/archive/model_cbow3.bin")

In [17]:
model2.wv.most_similar("salt")

[('kosher salt', 0.7338458299636841),
 ('kosher', 0.6353858113288879),
 ('coarse salt', 0.5166900753974915),
 ('ground sea salt', 0.5089148283004761),
 ('water', 0.4677097201347351),
 ('home zinfandel', 0.46338745951652527),
 ('tsbp tequila', 0.4532773494720459),
 ('kosher sea salt', 0.4468149244785309),
 ('plain yogurt', 0.4383079409599304),
 ('vegetable oli', 0.42021405696868896)]

In [16]:
full_data_recipe=pd.read_csv('/content/drive/MyDrive/archive/recipes_data.csv')

In [6]:
full_data_recipe['vectorEmbeddings'].head(
)

,vectorEmbeddings
0,[ 0.05987573 0.64265436 -0.3117954 -0.446682...
1,[ 2.2377365e+01 1.9507402e+01 -1.8372253e+01 ...
2,[ 9.208999 1.8398496 44.904434 7.58662...
3,[ 11.346859 -2.792293 -4.3263435 11.67...
4,[-54.600285 41.48448 -7.2843113 -24.23501...


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
NER_data=full_data_recipe['NER']
v2=TfidfVectorizer()
v2.fit(NER_data)

TfidfVectorizer()

In [19]:
from collections import defaultdict
max_idf = max(v2.idf_)
word_idf_weight = defaultdict(
    lambda: max_idf,
    [(word, v2.idf_[i]) for word, i in v2.vocabulary_.items()],
    )

In [20]:
import numpy as np

In [21]:
def doc_average(doc):
  mean = []
  for word in doc:
    if word in model2.wv.index_to_key:
      mean.append(model2.wv.get_vector(word) * word_idf_weight[word] )
    # else:
    #   print('not found')

  if not mean:
    return np.zeros(100)
  else:
    mean = np.array(mean).mean(axis=0)
    return mean

In [22]:
def doc_average_list(docs):
  return np.vstack([doc_average(doc) for doc in docs])

str

In [23]:
import ast

In [24]:
full_data_recipe['ingredients_list']=full_data_recipe['NER'].apply(ast.literal_eval)

In [26]:
myVectorEmbeddings=doc_average_list(full_data_recipe['ingredients_list'])

In [27]:
np.save('/content/drive/MyDrive/archive/embeddings.npy', myVectorEmbeddings)

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

In [136]:
inputs=['maize flour','water']
inputs_embedding=doc_average(inputs).reshape(1, -1)
cosine_similarity(inputs_embedding,myVectorEmbeddings[1121633].reshape(1,-1))



array([[0.21749537]])

In [33]:
from sklearn.neighbors import NearestNeighbors

In [137]:
new_inputs_embedding=doc_average(inputs).reshape(1,-1)

In [81]:
nn=NearestNeighbors(algorithm='ball_tree',metric='minkowski')
nbrs=nn.fit(myVectorEmbeddings)

In [138]:
distances,indices=nbrs.kneighbors(new_inputs_embedding,n_neighbors=10)


In [139]:
full_data_recipe.iloc[indices[0],][['ingredients_list','title']]

,ingredients_list,title
565918,[water],Disappearing Paint
542216,[water],Playdough
539605,[water],Laver-Seafood Soup(Traditional)
235834,[water],To Clean Silver
330791,[water],Boiled Water Soup
318766,[water],How To Boil Water Without Burning A Pan
19144,[water],Spice Snakin' Cake(Microwave)
327810,[water],Tropical Punch(100% Nutrasweet)
246292,[water],"""Surprise"" Spaghetti"
293435,[water],Kool-Aid


In [124]:
word_idf_weight['pancetta']

7.5039991258213705

In [123]:
distances

array([[34.16301737, 37.12896902, 38.68594034, 39.05376995, 39.23488892,
        39.23736814, 39.2624015 , 39.2624015 , 39.34398968, 39.86526072]])

array([[21.4348293 , 23.02712503, 23.1757938 ]])

In [62]:
inputs=['chicken thigh', 'onion', 'rice noodle', 'seaweed nori sheet', 'sesame', 'shallot', 'soy', 'spinach', 'star', 'tofu']
inputs_embedding=doc_average(inputs).reshape(1, -1)
inputs_embedding
def similarity_score(row):
  # print(type(row))
  if isinstance(row, np.ndarray):
    cosineSimilarity=cosine_similarity(row.reshape(1, -1),inputs_embedding)
    return cosineSimilarity[0][0]

  return 0

full_data_recipe['similarity_score']=full_data_recipe['newVectorEmbeddings'].apply(similarity_score)
# type(full_data_recipe['newVectorEmbeddings'][0])

In [63]:
index_max=full_data_recipe['similarity_score'].idxmax()
full_data_recipe.nlargest(10, ['similarity_score'])[['title','NER','similarity_score']]

,title,NER,similarity_score
1121633,Chicken Porridge,"[""chicken thigh"", ""spring onion"", ""rice"", ""chi...",0.871700
1979234,Teriyaki Burgers for Bento,"[""oil"", ""egg"", ""chicken breasts"", ""ginger"", ""o...",0.823440
2063794,Chawan-mushi (Steamed Egg Custard) in the Micr...,"[""kamaboko"", ""eggs"", ""shiitake mushrooms"", ""wa...",0.817734
2119419,Gingery Chicken and Tofu Hot Pot,"[""scallions"", ""water"", ""chicken thigh"", ""chick...",0.812155
998689,Udon Noodle Soup,"[""udon noodles"", ""water"", ""miso"", ""chili flake...",0.811446
2119664,Steamed Chicken with Whole Grain Mustard and P...,"[""tomatoes"", ""ponzu"", ""mirin"", ""chicken breast...",0.806999
546589,Modeling Dough,"[""dashi"", ""oil"", ""scallion"", ""tofu"", ""flour"", ...",0.802808
1661637,Mighty Miso Soup,"[""firm tofu"", ""shiitake mushrooms"", ""water"", ""...",0.796026
1663735,"Harissa chicken with lentil, buckwheat & aspar...","[""buckwheat"", ""chicken thigh"", ""spring onion"",...",0.795516
2042482,A Nagoya Speciality: Miso-Stewed Udon Noodles,"[""kamaboko"", ""egg"", ""sugar"", ""noodles"", ""green...",0.786269


In [59]:
inputs2=['chicken thigh', 'onion', 'rice noodle', 'seaweed nori sheet', 'sesame', 'shallot', 'soy', 'spinach', 'star', 'tofu']
def doc_averages(doc):
  mean = []
  for word in doc:
    if word in model2.wv.index_to_key:
      mean.append(model2.wv.get_vector(word) * word_idf_weight[word] )
    # else:
    #   print('not found')

  if not mean:
    return np.zeros(100)
  else:
    mean = np.array(mean).mean(axis=0)
    return mean
imputs=doc_averages(inputs2)
imputs

['chicken thigh', 'onion', 'rice noodle', 'seaweed nori sheet', 'sesame', 'shallot', 'soy', 'spinach', 'star', 'tofu']
chicken thigh
chicken thigh
onion
onion
rice noodle
rice noodle
seaweed nori sheet
sesame
sesame
shallot
shallot
soy
soy
spinach
spinach
star
tofu
tofu


array([-1.8708845e+00,  1.2833493e+00, -2.5319083e+00, -2.8101873e+00,
       -1.2082160e-03,  2.1517920e+00,  1.5408783e+00, -3.8182964e+00,
       -6.2509732e+00, -1.0938896e+00,  1.5587574e+00,  3.9689586e+00,
        2.3850256e-01,  1.1312706e+00, -5.9441309e+00,  2.3219566e+00,
       -3.0126097e+00, -9.6858108e-01,  4.8650088e+00,  1.3064632e+00,
       -1.2516620e+00,  2.7640138e+00, -4.2136788e+00, -2.0267572e+00,
       -7.3543715e-01, -6.2472196e+00,  5.5309591e+00,  4.0172935e-03,
        5.6225830e-01, -2.7031634e+00,  1.0764667e+00,  3.1939068e-01,
        6.8232865e+00, -1.1357197e+00,  1.9105098e+00, -6.6489595e-01,
        1.7720120e+00, -2.0964735e+00,  4.1515679e+00,  1.2895112e+00,
        7.1513448e+00, -1.6554652e+00, -1.3418214e+00, -2.7038937e+00,
       -4.6712768e-01,  1.1292090e+00,  4.5606413e+00, -5.7151995e+00,
       -6.4001031e+00,  1.5609162e+00,  9.9300444e-01,  6.3371480e-01,
       -1.6411142e+00,  2.9601884e+00,  1.1583433e+00, -3.8700166e+00,
      

In [53]:
model2.wv.index_to_key.index('spinach')


222

In [1]:
full_data_recipe.head()

NameError: name 'full_data_recipe' is not defined